In [7]:
import os
import numpy as np
from skimage.io import imread, imsave
from skimage.transform import rotate, resize
from tqdm import tqdm
import random

In [8]:
# Definir rutas
ruta_original = r'D:\Python\TESIS\CLUSTERS_C1_CALC_K\test0'
ruta_balanceado = r'D:\Python\TESIS\CLUSTERS_C1_CALC_K\test'

# Número máximo de imágenes en una de las clases
num_max_muestras = 42

# Crear carpeta de salida si no existe
if not os.path.exists(ruta_balanceado):
    os.makedirs(ruta_balanceado)

# Función para centrar la imagen en un lienzo de 170x170 píxeles
def centrar_imagen(imagen, tamano_lienzo=170):
    alto, ancho = imagen.shape
    # Si la imagen es más grande que el lienzo, la redimensionamos
    if alto > tamano_lienzo or ancho > tamano_lienzo:
        imagen = resize(imagen, (tamano_lienzo, tamano_lienzo), anti_aliasing=True)
        alto, ancho = imagen.shape
    
    pad_alto = (tamano_lienzo - alto) // 2
    pad_ancho = (tamano_lienzo - ancho) // 2
    imagen_centrada = np.pad(imagen, ((pad_alto, tamano_lienzo - alto - pad_alto), (pad_ancho, tamano_lienzo - ancho - pad_ancho)), mode='constant', constant_values=0)
    return imagen_centrada

# Función para convertir a uint8
def convertir_a_uint8(imagen):
    imagen_escala = (imagen - np.min(imagen)) / (np.max(imagen) - np.min(imagen))  # Normalizar la imagen entre 0 y 1
    imagen_uint8 = (imagen_escala * 255).astype(np.uint8)  # Convertir a valores entre 0 y 255 (uint8)
    return imagen_uint8

# Función para aplicar data augmentation
def augmentar_imagen(imagen):
    # Aplicar rotación aleatoria entre -15 y 15 grados
    angulo = random.uniform(-15, 15)
    imagen_rotada = rotate(imagen, angle=angulo, mode='edge')

    # Aplicar desplazamiento aleatorio
    desplazamiento_h = random.uniform(-10, 10)
    desplazamiento_v = random.uniform(-10, 10)
    imagen_desplazada = np.roll(imagen_rotada, int(desplazamiento_h), axis=1)
    imagen_desplazada = np.roll(imagen_desplazada, int(desplazamiento_v), axis=0)

    # Recortar el exceso generado por el desplazamiento
    imagen_desplazada = imagen_desplazada[:170, :170]

    # Aplicar zoom aleatorio
    factor_zoom = random.uniform(0.9, 1.1)
    imagen_zoom = resize(imagen_desplazada, (int(170 * factor_zoom), int(170 * factor_zoom)), anti_aliasing=True)
    
    # Centrar de nuevo en un lienzo 170x170 (en caso de que el zoom modifique el tamaño)
    imagen_final = centrar_imagen(imagen_zoom)
    
    return imagen_final

# Procesar cada clase
for carpeta in os.listdir(ruta_original):
    ruta_clase = os.path.join(ruta_original, carpeta)
    ruta_salida_clase = os.path.join(ruta_balanceado, carpeta)
    
    # Crear carpeta de salida para la clase si no existe
    if not os.path.exists(ruta_salida_clase):
        os.makedirs(ruta_salida_clase)
    
    # Obtener la lista de imágenes en la carpeta de la clase
    imagenes = os.listdir(ruta_clase)
    num_imagenes_actual = len(imagenes)
    
    # Cargar y centrar las imágenes existentes
    for imagen_nombre in tqdm(imagenes, desc=f'Procesando clase {carpeta}'):
        ruta_imagen = os.path.join(ruta_clase, imagen_nombre)
        imagen = imread(ruta_imagen, as_gray=True)
        imagen_centrada = centrar_imagen(imagen)
        
        # Convertir la imagen centrada a uint8
        imagen_centrada_uint8 = convertir_a_uint8(imagen_centrada)
        
        # Guardar la imagen centrada
        imsave(os.path.join(ruta_salida_clase, imagen_nombre), imagen_centrada_uint8)
    
    # Generar imágenes adicionales para balancear la clase
    if num_imagenes_actual < num_max_muestras:
        deficit = num_max_muestras - num_imagenes_actual
        for i in tqdm(range(deficit), desc=f'Balanceando clase {carpeta}'):
            # Seleccionar una imagen aleatoria de la clase para aplicar augmentación
            imagen_aleatoria = random.choice(imagenes)
            ruta_imagen = os.path.join(ruta_clase, imagen_aleatoria)
            imagen = imread(ruta_imagen, as_gray=True)
            
            # Aplicar augmentación y centrar en lienzo de 170x170
            imagen_aumentada = augmentar_imagen(imagen)
            
            # Convertir la imagen aumentada a uint8
            imagen_aumentada_uint8 = convertir_a_uint8(imagen_aumentada)
            
            # Guardar la imagen aumentada
            nombre_imagen_aumentada = f'aug_{i}_{imagen_aleatoria}'
            imsave(os.path.join(ruta_salida_clase, nombre_imagen_aumentada), imagen_aumentada_uint8)

print("Proceso de balanceo completado.")


Balanceando clase RADIOENLACE: 100%|███████████████████████████████████████████████████| 35/35 [00:00<00:00, 84.37it/s]

Proceso de balanceo completado.
